## 01. 라이브러리 로드

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pykrx import stock
import time

## 02. 데이터 로드

In [2]:
df = pd.read_csv('data/krx/KOSPI_KOSDAQ_log_returns_2011_2024.csv')

#### 날짜 컬럼 재정의

In [3]:
df['날짜'] = pd.to_datetime(df['날짜'])
df.set_index('날짜', inplace=True)

#### 분석 기간 정의
- 각 날짜가 어떤 기간에 속하는지 라벨링
- 각 기간에 해당하는 기간 시작일(4월 마지막 영업일)을 매핑

In [4]:
# 각 날짜가 어떤 '기간'에 속하는지 라벨링
period_year = np.where(df.index.month > 4, df.index.year, df.index.year - 1)
df['period_year'] = period_year

# 각 기간 해당하는 '기간 시작일'(4월 마지막 영업일)을 매핑
period_start_dates = {
    year: pd.to_datetime(f'{year}-04-01') + pd.offsets.BMonthEnd()
    for year in range(df['period_year'].min(), df['period_year'].max() + 1)
}
df['period_start_date'] = df['period_year'].map(period_start_dates)

## 03. 샤프지수 계산
- 기간별/티커별 일간 평균수익률과 일간변동성 계산
- 계산된 통계량 연율화
- 무위험수익률 생성 (2%)
- 샤프지수 계산 : (연간수익률 - 무위험수익률) / 연간변동성

In [5]:
grouped_stats = df.groupby(['period_start_date', '티커'])['로그수익률'].agg(['mean', 'std'])

# 계산된 통계량을 연율화
trading_days_per_year = 252
grouped_stats['연간수익률'] = grouped_stats['mean'] * trading_days_per_year
grouped_stats['연간변동성'] = grouped_stats['std'] * np.sqrt(trading_days_per_year)

risk_free_rate = 0.02

grouped_stats['샤프지수'] = (grouped_stats['연간수익률'] - risk_free_rate) / grouped_stats['연간변동성']

#### 결과에 필요한 컬럼 선택 및 재정의
- 인덱스를 컬럼으로 변환
- 종료일 컬럼 생성 (시작일로부터 1년 뒤 4월 말)
- 결과에 필요한 컬럼 선택 및 순서 정리

In [6]:
results_df = grouped_stats.reset_index()
results_df.rename(columns={'period_start_date': '시작일'}, inplace=True)

# '종료일' 컬럼 생성 (시작일로부터 1년 뒤 4월 말)
results_df['종료일'] = results_df['시작일'].apply(
    lambda date: pd.to_datetime(f'{date.year + 1}-04-01') + pd.offsets.BMonthEnd()
)

results_df = results_df[[
    '시작일', 
    '종료일', 
    '티커', 
    '연간수익률', 
    '연간변동성', 
    '샤프지수'
]]

## 04. 데이터 저장
- 회계년도 : 시작일에서 년도 추출해서 컬럼 생성
- 시작일, 종료일 컬럼 제거

In [7]:
results_df['회계년도'] = results_df['시작일'].dt.year
results_df.drop(columns=['시작일', '종료일'], inplace=True)
results_df.sort_values(by=['티커', '회계년도'], inplace=True)

#### 데이터 저장

In [8]:
results_df.to_excel("data/krx/sharpe.xlsx", index=False)